In [8]:
import xarray as xr
import netCDF4
import numpy as np
import pandas as pd
import aasi 
import pyeto as pt

In [84]:
#User-supplied

mapfile = '/glade/p/work/manab/ff/forcinggeneration/mappingislandpark.nc'
forcfile = '/glade/p/work/manab/ff/forcinggeneration/islandparkforc/ens_forc.PNW.0625.1980-1989.051.nc'
obsfile = '/glade/p/work/manab/ff/forcinggeneration/pilot_basin_data/isli_bor_flow_data.txt'
outfile = '/glade/p/work/manab/ff/islandpark/input/islandpark.nc'


def calcWeightedAvg(ncdat, varname):
    '''
    Calculates the weighted average 
    of any <varname> in the mapping file
    '''
    wAvg = sum(ncdat[varname]*ncdat['weight'])
    return(wAvg)

def calcPET(lat, time, tmin, tmax, tmean):
    '''
    Calculates Potential Evapotranspiration using 
    Hargreaves equation (Hargreaves and Samani, 1985) 
    '''
    
    latrad = pt.deg2rad(lat)                             #Latitude to radians
    dayofyear = pd.Series(time).dt.day.values
    etrad = []
    pet = []
    
    #Calculate ET radiation
    for x in np.nditer(dayofyear):
        soldec = pt.sol_dec(x)                           #Solar declination
        sha = pt.sunset_hour_angle(latrad, soldec)       #Sunset hour aingle
        ird = pt.inv_rel_dist_earth_sun(x)               #Inverse relative distance Earth-Sun
        etrad.append(pt.et_rad(latrad, soldec, sha, ird))#Extraterrestrial radiation
    
    #Calculate PET using hargreaves
    for x in range(0, len(etrad)):
        pet.append(pt.hargreaves(tmin[x], tmax[x], tmean[x], etrad[x]))
    
    pet = np.array(pet)
    return(pet)

def impqobs(file, starttime, endtime):
    '''
    Import Flow observation for the period
    '''
    df = pd.read_table(file, skiprows = 14, header = None, 
                    names = ['time', 'gaugeheight', 'resinflow', 'dailydisc', 'unregflow'])
    df['time'] = pd.to_datetime(data['time'])
    df = data.set_index(['time'])   #Set time as index
    df.loc[starttime:endtime]
    return(df)
    

if __name__ == '__main__':
    
    mapdat = xr.open_dataset(mapfile)
    forcdat = xr.open_dataset(forcfile)
    
    lat = calcWeightedAvg(mapdat, 'latitude')
    lon = calcWeightedAvg(mapdat, 'longitude')
    
    #Extracts the variables values from forcing file
    prcp = forcdat['prcp'].values
    tmax = forcdat['tmax'].values
    tmin = forcdat['tmin'].values
    time = forcdat['time'].values
    tmean = (tmax+tmin)/2
    
    pet = calcPET(lat, time, tmin, tmax, tmean)

    prcp = np.reshape(prcp,(prcp.shape[0],1,1))
    pet = np.reshape(pet,(pet.shape[0],1,1))
    tmean = np.reshape(tmax,(tmean.shape[0],1,1))
    
    #Create new netCDF file
    f = xr.Dataset({'pr': (['time','latitude', 'longitude'],  prcp),
                    'pet': (['time','latitude', 'longitude'],  pet),
                    'temp': (['time','latitude', 'longitude'],  tmean)
                   },
                   coords={'time': time,
                           'latitude':[lat],
                           'longitude':[lon]}
                  ) 
    f['q_obs'] = f['pr']*0+0.25
    


    

In [111]:
start = f['time'][0].values
impqobs(obsfile, start, start)

KeyError: 'time'

In [108]:
f['time'][0].values

numpy.datetime64('1980-01-01T00:00:00.000000000')

### 